In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Multi-area communication subspaces (repeat-split cvPCA) + alignment to ViT.

Outputs per area-pair:
  - communication σ spectrum with bootstrap CIs and shuffled null
  - effective shared dimensionality (rank) estimates
  - ViT alignment σ for A-comm and B-comm
  - saved plots and .npz bundles

Author: Maria + Pläku 🐾
"""

import os
import itertools
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scipy.special import softmax
from skbio.stats.composition import clr

# -----------------------------
# CONFIG
# -----------------------------
VIT_PATH    = '/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/LuckyMouse/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy'
AREAS_PATH  = '/home/maria/MITNeuralComputation/visualization/brain_area.npy'

AREAS       = ["VISp", "VISl", "VISrl", "VISal", "VISam", "VISpm"]   # edit as needed
PAIR_LIST   = list(itertools.combinations(AREAS, 2))                 # all unordered pairs

N_IMAGES, N_TRIALS = 118, 50
VAR_CUTOFF_VIT   = 0.90
VAR_CUTOFF_BRAIN = 0.90

K_OUTER     = 5       # stimulus-level CV folds
N_BOOT      = 500     # bootstrap resamples over TEST images
N_NULL      = 500     # shuffled-null resamples over TEST images
RNG_SEED    = 42
OUTDIR      = "results_comm_subspaces"
os.makedirs(OUTDIR, exist_ok=True)

rng = np.random.default_rng(RNG_SEED)

# -----------------------------
# Helpers
# -----------------------------
def fit_vit_pcs(vit_logits, var_cutoff=0.90, seed=RNG_SEED):
    Xv = softmax(np.asarray(vit_logits), axis=1)
    Xv = clr(Xv + 1e-12)
    pfull = PCA(random_state=seed).fit(Xv)
    ncomp = np.searchsorted(np.cumsum(pfull.explained_variance_ratio_), var_cutoff) + 1
    p = PCA(n_components=ncomp, random_state=seed).fit(Xv)
    Zv = p.transform(Xv)
    # stabilize scales a bit
    Zv = (Zv - Zv.mean(0)) / (Zv.std(0) + 1e-8)
    return Zv, p, int(ncomp)

def repeat_split_area(dat_area):
    """dat_area: (neurons, images, trials, time) -> (X_even, X_odd) as (neurons, images)."""
    idx = np.arange(N_TRIALS)
    even, odd = idx[::2], idx[1::2]
    Xe = dat_area[:, :, even, :].mean(axis=(2,3))
    Xo = dat_area[:, :, odd,  :].mean(axis=(2,3))
    return Xe, Xo

def brain_pcs_on_train(X_even, img_tr, X_odd, var_cutoff=0.90, seed=RNG_SEED):
    """
    Fit PCA on even repeats restricted to TRAIN IMAGES.
    Return even/odd projections for ALL images in the TRAIN basis.
    """
    pfull = PCA(random_state=seed).fit(X_even[:, img_tr].T)
    ncomp = np.searchsorted(np.cumsum(pfull.explained_variance_ratio_), var_cutoff) + 1
    p = PCA(n_components=ncomp, random_state=seed).fit(X_even[:, img_tr].T)

    Ze_all = (X_even.T - p.mean_) @ p.components_.T   # (N_IMAGES, ncomp)
    Zo_all = (X_odd.T  - p.mean_) @ p.components_.T
    # center across images for stability
    Ze_all -= Ze_all.mean(0); Zo_all -= Zo_all.mean(0)
    return Ze_all, Zo_all, p, int(ncomp)

def cvpca_comm(ZA, ZB):
    """Cross-covariance SVD on paired matrices (N×dA, N×dB) -> U_A, S, V_B."""
    C = (ZA.T @ ZB) / ZA.shape[0]
    UA, S, VBt = np.linalg.svd(C, full_matrices=False)
    return UA, S, VBt.T

def vit_alignment_sigmas(Zv, Z_comm):
    """SVD of cross-covariance Zv^T Z_comm / N, return singular values."""
    C = (Zv.T @ Z_comm) / Zv.shape[0]
    _, S, _ = np.linalg.svd(C, full_matrices=False)
    return S

def fdr_bh(p_vals, q=0.05):
    p = np.array(p_vals)
    m = len(p)
    order = np.argsort(p)
    thresh = q * (np.arange(1, m+1)) / m
    passed = p[order] <= thresh
    k = np.max(np.where(passed)[0]) + 1 if np.any(passed) else 0
    cutoff = p[order[k-1]] if k > 0 else None
    sig = p <= (cutoff if cutoff is not None else 0)
    return sig, cutoff

def effective_rank(real_mean, null_mean, ratio_thresh=1.2):
    """Count components whose mean σ is > ratio_thresh × null mean σ."""
    r = np.asarray(real_mean) / (np.asarray(null_mean) + 1e-12)
    idx = np.where(r > ratio_thresh)[0]
    return int(len(idx))

# -----------------------------
# Load data
# -----------------------------
with open(VIT_PATH, 'rb') as f:
    vit_logits = pickle.load(f)['natural_scenes']
Zv, vit_pca, vit_ncomp = fit_vit_pcs(vit_logits, var_cutoff=VAR_CUTOFF_VIT, seed=RNG_SEED)
print(f"ViT PCs covering {int(VAR_CUTOFF_VIT*100)}% variance: {vit_ncomp}")

dat = np.load(NEURAL_PATH, mmap_mode='r')               # (neurons, images*trials*time)
areas_vec = np.load(AREAS_PATH, allow_pickle=True)

# Pre-slice area matrices (once)
area_to_tensor = {}
for area in AREAS:
    mask = (areas_vec == area)
    dA = dat[mask]
    if dA.size == 0:
        print(f"[WARN] No data for area {area}; skipping.")
        continue
    n_neu, n_total = dA.shape
    n_time = n_total // (N_IMAGES * N_TRIALS)
    dA = dA.reshape(n_neu, N_IMAGES, N_TRIALS, n_time)
    area_to_tensor[area] = dA

# -----------------------------
# Outer stimulus K-fold loop per pair
# -----------------------------
kf = KFold(n_splits=K_OUTER, shuffle=True, random_state=RNG_SEED)

for (A, B) in PAIR_LIST:
    if A not in area_to_tensor or B not in area_to_tensor:
        continue

    print(f"\n=== Communication subspace: {A} ↔ {B} ===")

    dA = area_to_tensor[A]
    dB = area_to_tensor[B]

    # Repeat split
    A_even, A_odd = repeat_split_area(dA)
    B_even, B_odd = repeat_split_area(dB)

    stim_idx = np.arange(N_IMAGES)

    # Collect fold-wise spectra and alignment
    fold_S_list = []
    fold_SvA_list, fold_SvB_list = [], []

    # For bootstrap/null we’ll pool test indices from each fold’s data arrays
    # and resample within fold (keeps fold size constant).
    for fold, (idx_tr, idx_te) in enumerate(kf.split(stim_idx), 1):
        img_tr = stim_idx[idx_tr]
        img_te = stim_idx[idx_te]

        # Fit PCA per area on TRAIN images (even repeats), project even/odd for ALL images
        A_Ze_all, A_Zo_all, A_pca, A_n = brain_pcs_on_train(A_even, img_tr, A_odd, VAR_CUTOFF_BRAIN, RNG_SEED)
        B_Ze_all, B_Zo_all, B_pca, B_n = brain_pcs_on_train(B_even, img_tr, B_odd, VAR_CUTOFF_BRAIN, RNG_SEED)

        # Use TEST images and TEST repeats for cvPCA (no leakage): choose odd as "test view"
        A_te = A_Zo_all[img_te]  # (n_test, dA)
        B_te = B_Zo_all[img_te]  # (n_test, dB)

        # Communication cvPCA on test split
        UA, S_comm, VB = cvpca_comm(A_te, B_te)
        fold_S_list.append(S_comm)

        # Build k-dimensional communication scores to align with ViT
        # choose k as min(10, rank), can also revisit with null later
        k = min(10, len(S_comm))
        A_comm_scores = A_Zo_all @ UA[:, :k]   # (N_IMAGES, k)
        B_comm_scores = B_Zo_all @ VB[:, :k]   # (N_IMAGES, k)

        # ViT alignment on TEST images
        SvA = vit_alignment_sigmas(Zv[img_te], A_comm_scores[img_te])
        SvB = vit_alignment_sigmas(Zv[img_te], B_comm_scores[img_te])
        fold_SvA_list.append(SvA)
        fold_SvB_list.append(SvB)

        # -------- Bootstrap over test images (store once per fold) --------
        S_boot = np.zeros((N_BOOT, len(S_comm)))
        S_null = np.zeros((N_NULL, len(S_comm)))

        for b in range(N_BOOT):
            idx = rng.integers(0, len(img_te), len(img_te))
            UA_b, S_b, VB_b = cvpca_comm(A_te[idx], B_te[idx])
            S_boot[b, :len(S_b)] = S_b

        for b in range(N_NULL):
            # Shuffle pairing between A and B on TEST images
            perm = rng.permutation(len(img_te))
            UA_n, S_n, VB_n = cvpca_comm(A_te, B_te[perm])
            S_null[b, :len(S_n)] = S_n

        # Aggregate fold-wise (means; CIs can be saved too)
        S_mean = S_boot.mean(0)
        S_null_mean = S_null.mean(0)

        # Effective rank for this fold
        rank_eff = effective_rank(S_mean, S_null_mean, ratio_thresh=1.2)

        # Save per-fold plot
        x = np.arange(1, len(S_mean)+1)
        plt.figure(figsize=(7,5))
        plt.plot(x, S_mean, 'o-', label='Real (bootstrap mean)')
        lo, hi = np.percentile(S_boot, [2.5, 97.5], axis=0)
        plt.fill_between(x, lo, hi, alpha=0.2)

        plt.plot(x, S_null_mean, 'o--', label='Null (mean)')
        lo_n, hi_n = np.percentile(S_null, [2.5, 97.5], axis=0)
        plt.fill_between(x, lo_n, hi_n, alpha=0.2, color='gray')

        plt.title(f"{A}↔{B} comm (fold {fold})  rank_eff≈{rank_eff}")
        plt.xlabel("cvPCA component"); plt.ylabel("σ (shared)")
        plt.legend(); plt.tight_layout()
        plt.savefig(os.path.join(OUTDIR, f"{A}_{B}_fold{fold}_comm_spectrum.png"), dpi=150)
        plt.close()

        # Save fold bundle
        np.savez(os.path.join(OUTDIR, f"{A}_{B}_fold{fold}_comm_bundle.npz"),
                 S_boot=S_boot, S_null=S_null, S_mean=S_mean, S_null_mean=S_null_mean,
                 UA=UA, VB=VB, A_n=A_n, B_n=B_n,
                 SvA=SvA, SvB=SvB, rank_eff=rank_eff,
                 img_tr=img_tr, img_te=img_te)

    # -------- Across-fold summary --------
    # pad spectra to same length to average (use min length)
    L = min([len(s) for s in fold_S_list])
    S_stack = np.stack([s[:L] for s in fold_S_list], axis=0)
    S_comm_mean_across_folds = S_stack.mean(0)

    # ViT alignment: take first min length there, too
    L_vA = min(len(sv) for sv in fold_SvA_list)
    L_vB = min(len(sv) for sv in fold_SvB_list)
    SvA_stack = np.stack([sv[:L_vA] for sv in fold_SvA_list], axis=0)
    SvB_stack = np.stack([sv[:L_vB] for sv in fold_SvB_list], axis=0)
    SvA_mean = SvA_stack.mean(0)
    SvB_mean = SvB_stack.mean(0)

    # Quick printout
    print(f"Top-5 mean σ (comm): {np.round(S_comm_mean_across_folds[:5], 3)}")
    print(f"Top-5 mean σ (ViT↔{A}-comm): {np.round(SvA_mean[:5], 3)}")
    print(f"Top-5 mean σ (ViT↔{B}-comm): {np.round(SvB_mean[:5], 3)}")

    # Save summary
    np.savez(os.path.join(OUTDIR, f"{A}_{B}_summary.npz"),
             S_comm_mean=S_comm_mean_across_folds,
             SvA_mean=SvA_mean, SvB_mean=SvB_mean)

    # One compact across-fold spectrum plot (no null bands here)
    x = np.arange(1, len(S_comm_mean_across_folds)+1)
    plt.figure(figsize=(7,5))
    for s in fold_S_list:
        plt.plot(x[:len(s[:L])], s[:L], color='lightsteelblue', alpha=0.4)
    plt.plot(x, S_comm_mean_across_folds, 'o-', color='tab:blue', label='mean across folds')
    plt.title(f"{A}↔{B} communication σ (across folds)")
    plt.xlabel("cvPCA component"); plt.ylabel("σ (shared)")
    plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(OUTDIR, f"{A}_{B}_across_folds_comm_spectrum.png"), dpi=150)
    plt.close()


ViT PCs covering 90% variance: 44

=== Communication subspace: VISp ↔ VISl ===
Top-5 mean σ (comm): [1.416 0.969 0.813 0.58  0.467]
Top-5 mean σ (ViT↔VISp-comm): [2.133 1.605 1.345 1.183 1.016]
Top-5 mean σ (ViT↔VISl-comm): [1.758 1.485 1.256 1.054 0.937]

=== Communication subspace: VISp ↔ VISrl ===
Top-5 mean σ (comm): [0.485 0.239 0.151 0.126 0.098]
Top-5 mean σ (ViT↔VISp-comm): [2.131 1.6   1.338 1.176 0.985]
Top-5 mean σ (ViT↔VISrl-comm): [0.744 0.369 0.263 0.223 0.205]

=== Communication subspace: VISp ↔ VISal ===
Top-5 mean σ (comm): [0.942 0.66  0.54  0.386 0.305]
Top-5 mean σ (ViT↔VISp-comm): [2.125 1.604 1.343 1.181 1.002]
Top-5 mean σ (ViT↔VISal-comm): [1.344 1.027 0.883 0.734 0.654]

=== Communication subspace: VISp ↔ VISam ===
Top-5 mean σ (comm): [0.591 0.408 0.353 0.236 0.185]
Top-5 mean σ (ViT↔VISp-comm): [2.122 1.598 1.335 1.175 1.003]
Top-5 mean σ (ViT↔VISam-comm): [0.925 0.7   0.589 0.466 0.387]

=== Communication subspace: VISp ↔ VISpm ===
Top-5 mean σ (comm): [0.92